In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
import google.generativeai as genai
import gradio as gr

In [2]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY', 'your-key-if-not-using-env')

In [3]:
# Initialize clients
openai_client = OpenAI()
claude_client = anthropic.Anthropic(api_key=os.environ['ANTHROPIC_API_KEY'])
genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

In [4]:
# Model configurations
OPENAI_MODEL = "gpt-4o-mini"
CLAUDE_MODEL = "claude-3-5-sonnet-20240620"
GEMINI_MODEL = "gemini-2.0-flash-exp"

In [5]:
def system_prompt_for_docstring(language):
    """
    Returns a language-appropriate system prompt for docstring generation.

    Args:
        language (str): Programming language (e.g. 'python', 'javascript').

    Returns:
        str: System prompt string. Defaults to Python prompt if language is unrecognised.
    """
    prompts = {
        "python": (
            "You are a Python documentation expert. "
            "Follow PEP 257 and Google docstring style. "
            "Include type hints, parameters, return values, raised exceptions, and edge cases. "
            "Add inline comments only for complex logic."
        ),
        "javascript": (
            "You are a JavaScript/TypeScript documentation expert. "
            "Follow JSDoc standards. Include @param, @returns, and @throws tags. "
            "Use modern ES6+ examples."
        ),
        "java": (
            "You are a Java documentation expert. "
            "Follow Javadoc standards. Include @param, @return, and @throws tags "
            "for all public methods and classes."
        ),
        "cpp": (
            "You are a C++ documentation expert. "
            "Follow Doxygen standards. Document parameters, return values, exceptions, "
            "and memory management considerations."
        ),
        "go": (
            "You are a Go documentation expert. "
            "Follow Go documentation conventions. Document all exported functions and types."
        ),
        "rust": (
            "You are a Rust documentation expert. "
            "Follow Rust doc conventions. Document safety considerations and include "
            "examples in doc comments."
        ),
    }
    return prompts.get(language.lower(), prompts["python"])

In [6]:
def user_prompt_for_docstring(code, language):
    """
    Builds the user-facing prompt for docstring generation.

    Args:
        code (str): Source code to document.
        language (str): Programming language of the code.

    Returns:
        str: Formatted prompt string ready to send to an LLM.
    """
    return (
        f"Please document the following {language} code with comprehensive docstrings and comments.\n\n"
        "Requirements:\n"
        "- Add docstrings with: description, parameters (with types), return values, exceptions raised, and any important notes.\n"
        "- Add inline comments for complex algorithms, non-obvious choices, performance considerations, and edge cases.\n"
        "- Return ONLY the documented code. Do not include any explanation before or after.\n\n"
        f"Code to document:\n\n{code}"
    )

In [7]:
def strip_code_fences(text):
    """
    Removes Markdown code fences from a string.

    Args:
        text (str): Text potentially containing code fences.

    Returns:
        str: Text with all code fence markers removed.
    """
    for fence in ['```python', '```javascript', '```java', '```cpp', '```go', '```rust', '```']:
        text = text.replace(fence, '')
    return text

In [8]:
def stream_docstring_gpt(code, language):
    """
    Generate docstrings using OpenAI GPT with streaming.

    Args:
        code (str): Source code to document.
        language (str): Programming language.

    Yields:
        str: Cumulative documented code as it streams in.
    """
    stream = openai_client.chat.completions.create(
        model=OPENAI_MODEL,
        messages=[
            {"role": "system", "content": system_prompt_for_docstring(language)},
            {"role": "user", "content": user_prompt_for_docstring(code, language)},
        ],
        stream=True,
        temperature=0.3,
    )

    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield strip_code_fences(reply)

In [9]:
def stream_docstring_claude(code, language):
    """
    Generate docstrings using Anthropic Claude with streaming.

    Args:
        code (str): Source code to document.
        language (str): Programming language.

    Yields:
        str: Cumulative documented code as it streams in.
    """
    reply = ""
    with claude_client.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=4096,
        system=system_prompt_for_docstring(language),
        messages=[{"role": "user", "content": user_prompt_for_docstring(code, language)}],
    ) as stream:
        for text in stream.text_stream:
            reply += text
            yield strip_code_fences(reply)

In [10]:
def stream_docstring_gemini(code, language):
    """
    Generate docstrings using Google Gemini with streaming.

    Args:
        code (str): Source code to document.
        language (str): Programming language.

    Yields:
        str: Cumulative documented code as it streams in.
    """
    model = genai.GenerativeModel(
        model_name=GEMINI_MODEL,
        system_instruction=system_prompt_for_docstring(language),
    )

    response = model.generate_content(
        user_prompt_for_docstring(code, language),
        stream=True,
        generation_config=genai.types.GenerationConfig(
            temperature=0.3,
            max_output_tokens=4096,
        ),
    )

    reply = ""
    for chunk in response:
        if chunk.text:
            reply += chunk.text
            yield strip_code_fences(reply)

In [11]:
def generate_docstring(code, language, model):
    """
    Route a docstring generation request to the appropriate model.

    Args:
        code (str): Source code to document.
        language (str): Programming language.
        model (str): One of 'GPT', 'Claude', or 'Gemini'.

    Yields:
        str: Progressively generated documented code.

    Raises:
        ValueError: If an unrecognised model name is provided.
    """
    if not code.strip():
        yield "Please enter some code to document."
        return

    dispatch = {
        "GPT": stream_docstring_gpt,
        "Claude": stream_docstring_claude,
        "Gemini": stream_docstring_gemini,
    }

    if model not in dispatch:
        yield f"Error: Unknown model '{model}'. Choose from: {', '.join(dispatch.keys())}."
        return

    try:
        yield from dispatch[model](code, language)
    except Exception as e:
        yield f"Error: {e}\n\nPlease check your API keys in the .env file."

In [12]:
# Example code snippets for testing
SAMPLE_PYTHON_CODE = """
def calculate_pi(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

class DataProcessor:
    def __init__(self, data):
        self.data = data
        self.processed = False

    def process(self, threshold=0.5):
        if not self.data:
            raise ValueError("No data to process")
        result = [x for x in self.data if x > threshold]
        self.processed = True
        return result
"""

SAMPLE_JAVASCRIPT_CODE = """
function calculateSum(numbers) {
    return numbers.reduce((acc, num) => acc + num, 0);
}

class UserManager {
    constructor(users) {
        this.users = users;
    }

    findByAge(minAge, maxAge) {
        return this.users.filter(user =>
            user.age >= minAge && user.age <= maxAge
        );
    }
}
"""

SAMPLE_JAVA_CODE = """
public class Calculator {
    private double result;

    public Calculator() {
        this.result = 0.0;
    }

    public double add(double a, double b) {
        result = a + b;
        return result;
    }

    public double divide(double a, double b) {
        if (b == 0) {
            throw new ArithmeticException("Division by zero");
        }
        result = a / b;
        return result;
    }
}
"""

In [ ]:
def load_example(example_name):
    """Return the example code string matching the given example name."""
    examples = {
        "Python Example": SAMPLE_PYTHON_CODE,
        "JavaScript Example": SAMPLE_JAVASCRIPT_CODE,
        "Java Example": SAMPLE_JAVA_CODE,
        "Custom": "",
    }
    return examples.get(example_name, "")


with gr.Blocks(theme=gr.themes.Soft()) as ui:
    gr.Markdown("# AI Docstring Generator")
    gr.Markdown("Automatically generate comprehensive docstrings and comments for your code.")

   

    with gr.Row():
        with gr.Column():
            code_input = gr.Textbox(
                label="Input Code",
                value=SAMPLE_PYTHON_CODE,
                lines=20,
                placeholder="Paste your code here...",
            )
            generate_btn = gr.Button("Generate Docstrings", variant="primary")
        with gr.Column():
            code_output = gr.Textbox(
                label="Documented Code",
                lines=20,
            )
    with gr.Row():
        language_dropdown = gr.Dropdown(
            choices=["Python", "JavaScript", "Java", "C++", "Go", "Rust"],
            label="Programming Language",
            value="Python",
        )
        model_dropdown = gr.Dropdown(
            choices=["GPT", "Claude", "Gemini"],
            label="AI Model",
            value="GPT",
        )
        example_dropdown = gr.Dropdown(
            choices=["Python Example", "JavaScript Example", "Java Example", "Custom"],
            label="Load Example",
            value="Python Example",
        )

    example_dropdown.change(fn=load_example, inputs=[example_dropdown], outputs=[code_input])
    generate_btn.click(
        fn=generate_docstring,
        inputs=[code_input, language_dropdown, model_dropdown],
        outputs=[code_output],
    )

In [17]:
ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.
